In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import time
import os

from panda_rod_utils import *

In [ ]:
plt.rcParams['figure.dpi'] = 500

In [ ]:
if_distance = False
if_sigma = False
if_video = True
if_make_new_dir = False
if_component = True

In [ ]:
time_now = time.strftime("%Y-%m-%d___%H-%M-%S", time.localtime())
print(time_now)

movie_path = '/home/zhux/robust-safe-set/src/pybullet-dynamics/panda_rod_env/movies/'
if if_make_new_dir:
    true_save_path = movie_path + time_now + '/'
    os.mkdir(true_save_path)
else:    
    true_save_path = movie_path + list(os.walk(movie_path))[0][1][-1] + '/'

In [ ]:
print(true_save_path)

In [ ]:
with open('/home/zhux/robust-safe-set/src/pybullet-dynamics/panda_rod_env/data/monitor.pkl', 'rb') as file:
    data = pickle.load(file)

In [ ]:
data.keys()

In [ ]:
# ssa_args
if 'ssa_args' in data.keys():
    for dic in data['ssa_args']:
        for key, value in dic.items():
            tmp = type(value)
            if tmp is int or tmp is float or tmp is bool:
                print(f'{key}: {value}')
            elif tmp is list:
                if len(value) < 10:
                    print(f'{key}: {value}')

In [ ]:
# env_args
if 'env_args' in data.keys():
    for dic in data['env_args']:
        for key, value in dic.items():
            tmp = type(value)
            if tmp is int or tmp is float or tmp is bool:
                print(f'{key}: {value}')
            elif tmp is list:
                if len(value) < 10:
                    print(f'{key}: {value}')

In [ ]:
# f and f_nn
if if_distance and 'f' in data.keys() and 'f_nn' in data.keys():
    f = to_np(data['f'])
    f_nn = to_np(data['f_nn'])
    dis = get_L2_norm(f, f_nn)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
if if_distance and 'f' in data.keys() and 'f_nn' in data.keys() and 'f_res' in data.keys():
    f = to_np(data['f'])
    f_nn = to_np(data['f_nn'])
    f_res = to_np(data['f_res'])
    f_nn_new = f_nn + f_res
    dis = get_L2_norm(f, f_nn_new)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
# g and g_nn
if if_distance and 'g' in data.keys() and 'g_nn' in data.keys():
    g = to_np(data['g'])
    g_nn = to_np(data['g_nn']).reshape(g.shape)
    dis = get_L2_norm(g, g_nn)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
if if_distance and 'g' in data.keys() and 'g_nn' in data.keys() and 'g_res' in data.keys():
    g = to_np(data['g'])
    g_nn = to_np(data['g_nn']).reshape(g.shape)
    g_res = to_np(data['g_res'])
    g_nn_new = g_nn + g_res
    dis = get_L2_norm(g, g_nn_new)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
# dot_Xr and dot_Xr_nn
if if_distance and 'dot_Xr' in data.keys():
    dot_Xr = to_np(data['dot_Xr'])
    u = to_np(data['u'])
    dis = []
    for i in range(dot_Xr.shape[0]):
        dot_Xr_nn = f_nn[i].reshape(-1, 1) + g_nn[i].reshape(18, 7) @ u[i].reshape(-1, 1)
        tmp = np.linalg.norm(dot_Xr[i] - dot_Xr_nn.reshape(-1,))
        dis.append(tmp)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
# largest singular value in f_nn_sigma
if if_sigma and 'f_nn_sigma' in data.keys():
    f_nn_sigma = to_np(data['f_nn_sigma'])
    largest_singular_value = get_largest_singular_value(f_nn_sigma)
    plt.plot(np.arange(len(largest_singular_value)), largest_singular_value)

In [ ]:
# largest singular value in g_nn_sigma
if if_sigma and 'g_nn_sigma' in data.keys():
    g_nn_sigma = to_np(data['g_nn_sigma'])
    largest_singular_value = get_largest_singular_value(g_nn_sigma)
    plt.plot(np.arange(len(largest_singular_value)), largest_singular_value)

In [ ]:
# LfP and LfP_nn
if if_distance and 'LfP' in data.keys() and 'LfP_nn' in data.keys():
    LfP = to_np(data['LfP'])
    LfP_nn = to_np(data['LfP_nn'])
    dis = get_L2_norm(LfP, LfP_nn)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
# LgP and LgP_nn
if if_distance and 'LgP' in data.keys() and 'LgP_nn' in data.keys():
    LgP = to_np(data['LgP'])
    LgP_nn = to_np(data['LgP_nn'])
    
    dis = get_L2_norm(LgP, LgP_nn)
    plt.plot(np.arange(len(dis)), dis)

In [ ]:
# largest singular value in LfP_nn_sigma
if if_sigma and 'LfP_nn_sigma' in data.keys():
    LfP_nn_sigma = data['LfP_nn_sigma']
    LfP_nn_sigma = to_np(LfP_nn_sigma)
    largest_singular_value = get_largest_singular_value(LfP_nn_sigma)
    plt.plot(np.arange(len(largest_singular_value)), largest_singular_value)

In [ ]:
# largest singular value in LgP_nn_sigma
if if_sigma and 'LgP_nn_sigma' in data.keys():
    LgP_nn_sigma = data['LgP_nn_sigma']
    LgP_nn_sigma = to_np(LgP_nn_sigma)
    largest_singular_value = get_largest_singular_value(LgP_nn_sigma)
    plt.plot(np.arange(len(largest_singular_value)), largest_singular_value)

In [ ]:
# record f and (f - f_nn) in a time interval
if if_video and 'f' in data.keys() and 'f_nn' in data.keys():
    f = to_np(data['f'])
    f_nn = to_np(data['f_nn'])
    images = get_images_of_array({'f': f, 'f_res': f - f_nn})
    video_record(true_save_path + 'f_and_f_res.mp4', images)

In [ ]:
# record g and (g - g_nn) in a time interval
if if_video and 'g' in data.keys() and 'g_nn' in data.keys():
    g = to_np(data['g'])
    g = g.reshape(g.shape[0], -1)
    g_nn = to_np(data['g_nn'])
    g_nn = g_nn.reshape(g_nn.shape[0], -1)
    images = get_images_of_array({'g': g, 'g_res': g - g_nn})
    video_record(true_save_path + 'g_and_g_res.mp4', images)

In [ ]:
# record dot_Xr and (dot_Xr - dot_Xr_nn) in a time interval
if if_video and 'dot_Xr' in data.keys() and 'f_nn' in data.keys() and 'g_nn' in data.keys():
     dot_Xr = to_np(data['dot_Xr'])
     f_nn = to_np(data['f_nn'])
     g_nn = to_np(data['g_nn']).reshape(f_nn.shape[0], f_nn.shape[1], -1)
     u = to_np(data['u']).reshape(f_nn.shape[0], -1, 1)
     dot_Xr_nn = f_nn + to_np(g_nn @ u)
     images = get_images_of_array({'dot_Xr': dot_Xr, 'dot_Xr_res': dot_Xr - dot_Xr_nn})
     video_record(true_save_path + 'dot_Xr_and_dot_Xr_res.mp4', images)

In [ ]:
# draw components of f and (f - f_nn)
if if_component and  'f' in data.keys() and 'f_nn' in data.keys():
    f = to_np(data['f'])
    f_nn = to_np(data['f_nn'])
    plot_images_of_components({'f': f, 'f_res': f - f_nn}, ids=np.arange(f.shape[1]), plt_array=[3, 6], figsize=(30, 10))

In [ ]:
# draw components of dot_Xr and (dot_Xr - dot_Xr_nn)
if if_component and 'dot_Xr' in data.keys() and 'f_nn' in data.keys() and 'g_nn' in data.keys():
    dot_Xr = to_np(data['dot_Xr'])
    f_nn = to_np(data['f_nn'])
    g_nn = to_np(data['g_nn']).reshape(f_nn.shape[0], f_nn.shape[1], -1)
    u = to_np(data['u']).reshape(f_nn.shape[0], -1, 1)
    dot_Xr_nn = f_nn + to_np(g_nn @ u)
    plot_images_of_components({'dot_Xr': dot_Xr, 'dot_Xr_res': dot_Xr - dot_Xr_nn}, ids=np.arange(dot_Xr.shape[1]), plt_array=[3, 6], figsize=(30, 10))

In [ ]:
# record dot_Xr and (dot_Xr - dot_Xr_nn) in a time interval
# if f is true
if if_video and 'dot_Xr' in data.keys() and 'f_nn' in data.keys() and 'g_nn' in data.keys():
     dot_Xr = to_np(data['dot_Xr'])
     f_nn = to_np(data['f'])
     g_nn = to_np(data['g_nn']).reshape(f_nn.shape[0], f_nn.shape[1], -1)
     u = to_np(data['u']).reshape(f_nn.shape[0], -1, 1)
     dot_Xr_nn = f_nn + to_np(g_nn @ u)
     images = get_images_of_array({'dot_Xr': dot_Xr, 'dot_Xr_res': dot_Xr - dot_Xr_nn})
     video_record(true_save_path + 'dot_Xr_and_dot_Xr_res_g_true.mp4', images)